In [38]:
import pandas as pd
import numpy as np
import requests
import time

In [3]:
KEY = 'Null for public'

# Figure out how to pull coordinate data using MapQuest API

In [22]:
response = requests.get('http://open.mapquestapi.com/geocoding/v1/address?key=' + KEY + '&location=Washington,DC')
test_str = 'http://open.mapquestapi.com/geocoding/v1/address?key=6dPARSfkaUHACBkYJLbT7N11d2fKQg9O&location=1369%20Cambridge%20Street,%20Cambridge%20MA,'
null_lat = 39.78373
null_lng = -100.445882

In [32]:
# Get coordinates from request
response.json()['results'][0]['locations'][0]['latLng']#['lat']

{'lat': 38.894955, 'lng': -77.036646}

In [33]:
def get_coords(address):
    response = requests.get('http://open.mapquestapi.com/geocoding/v1/address?key=' + KEY + '&location=' + address + ', Cambridge MA,')

    lat = response.json()['results'][0]['locations'][0]['latLng']['lat']
    lng = response.json()['results'][0]['locations'][0]['latLng']['lng']

    return lat, lng


# Clean and parse bussiness data in order to pass to API

In [50]:
businesses = pd.read_csv('Open_and_Closed_Businesses_During_Covid-19_Pandemic.csv')
businesses.head()

,Name,Address,Storefront Sales Status,Sales Options,Grocery Sales,Phone,Website,Facebook,Twitter,Instagram,LastModified
0,1369 Coffee House,NaN,Open,"Delivery, Take Out",yes,757 Mass Ave,NaN,https://www.facebook.com/1369Coffeehouse/,https://twitter.com/1369coffeehouse?ref_src=tw...,https://www.instagram.com/1369coffeehouse/?hl=en,2020-06-16T17:08:30Z
1,1369 Coffee House,NaN,Open,"Delivery, Take Out, Outdoor Dining",NaN,1369 Cambridge Street,NaN,https://www.facebook.com/1369Coffeehouse/,https://twitter.com/1369coffeehouse,https://www.instagram.com/1369coffeehouse/?hl=en,2020-07-10T15:24:36Z
2,2 4-1 Optical,NaN,Open,NaN,NaN,1128 Massachusetts Ave,NaN,https://www.facebook.com/241optical/,NaN,NaN,2020-06-16T17:09:04Z
3,5th Street Laundromat,NaN,Open,NaN,NaN,58 5th Street,NaN,NaN,NaN,NaN,2020-03-22T12:45:32Z
4,7-Eleven,NaN,Open,NaN,NaN,All locations are open,NaN,NaN,NaN,NaN,2020-06-09T17:16:46Z


In [51]:
# Remove bussinesses wrongly encoded as a group (only 7eleven gas stations in this case)
#businesses = businesses[businesses.Phone != 'All locations are open']
#businesses = businesses[businesses.Phone != 'All locations']

# Replace all Mass Ave abbreviations to Massachusetts Ave (The API doesn't recognize Mass Ave)
businesses.Phone = [str(x).replace('Mass ', 'Massachusetts ') for x in businesses.Phone]
businesses.Phone = [str(x).replace('Mass. ', 'Massachusetts ') for x in businesses.Phone]
businesses.Phone = [str(x).replace('JFK ', 'John F. Kennedy ') for x in businesses.Phone]

businesses.head()

,Name,Address,Storefront Sales Status,Sales Options,Grocery Sales,Phone,Website,Facebook,Twitter,Instagram,LastModified
0,1369 Coffee House,NaN,Open,"Delivery, Take Out",yes,757 Massachusetts Ave,NaN,https://www.facebook.com/1369Coffeehouse/,https://twitter.com/1369coffeehouse?ref_src=tw...,https://www.instagram.com/1369coffeehouse/?hl=en,2020-06-16T17:08:30Z
1,1369 Coffee House,NaN,Open,"Delivery, Take Out, Outdoor Dining",NaN,1369 Cambridge Street,NaN,https://www.facebook.com/1369Coffeehouse/,https://twitter.com/1369coffeehouse,https://www.instagram.com/1369coffeehouse/?hl=en,2020-07-10T15:24:36Z
2,2 4-1 Optical,NaN,Open,NaN,NaN,1128 Massachusetts Ave,NaN,https://www.facebook.com/241optical/,NaN,NaN,2020-06-16T17:09:04Z
3,5th Street Laundromat,NaN,Open,NaN,NaN,58 5th Street,NaN,NaN,NaN,NaN,2020-03-22T12:45:32Z
4,7-Eleven,NaN,Open,NaN,NaN,All locations are open,NaN,NaN,NaN,NaN,2020-06-09T17:16:46Z


# Finally get coordinates for each address (or at least try to)

In [52]:
nrow = businesses.shape[0]
lats = np.zeros(nrow)
lngs = np.zeros(nrow)

for i in range(nrow):
    lat, lng = get_coords(businesses.Phone.loc[i])
    lats[i] = lat
    lngs[i] = lng
    time.sleep(0.1)

businesses.insert(1, "Latitude", lats)
businesses.insert(2, "Longitude", lngs)

In [56]:
# print number of rows that weren't recognized be the API
print((businesses[businesses.Longitude == -100.445882].Phone).shape[0])

65


We only lost 65 rows out of 800+ businesses, not too bad!

# Save dataframe to csv

In [59]:
# drop rows that were not recognized by API
geocoded_businesses = businesses[businesses.Longitude != -100.445882]

geocoded_businesses.to_csv('geocoded_businesses.csv')